### Try to correct the gene names and gene id in pseudobulk, DE results, and summary statistics

In [1]:
import anndata as ad
import pandas as pd

In [2]:
datadir = '/Users/rzhu/Gladstone Dropbox/Ronghui Zhu/GRNPerturbSeq/3_expts/processed_data/CD4i_final'
experiment_name = 'CD4i_final'
adata_de = ad.read_h5ad(datadir + f'/DE_results_all_confounders/{experiment_name}.merged_DE_results.h5ad', backed='r')
adata_pb = ad.read_h5ad(datadir + '/CD4i_final_merged.DE_pseudobulk.h5ad', backed='r')
de_summary_stats = pd.read_csv(datadir + f'/DE_results_all_confounders/DE_summary_stats_per_target.csv')

In [31]:
name_df = pd.read_parquet('./genome/GWCD4CRISPRi_targeting_sgRNA_list_updated.parquet')

### Correct names in DE analysis

In [34]:
# First identify gene id that are not in the reference id list, correct their gene id
de_oldname_with_wrongid = adata_de[adata_de.obs.target_contrast.isin(list(set(adata_de.obs.target_contrast) - set(name_df.gene_id)))].obs.target_contrast_gene_name.unique()
de_oldname_to_newgeneid = name_df[name_df.target_gene_name_from_sgRNA.isin(de_oldname_with_wrongid)][['target_gene_name_from_sgRNA', 'gene_id']]
de_oldname_to_newgeneid.drop_duplicates(inplace=True)
de_oldname_to_newgeneid = de_oldname_to_newgeneid.set_index('target_gene_name_from_sgRNA')
adata_de.obs['target_contrast_gene_name_corrected'] = adata_de.obs['target_contrast_gene_name'].astype(str)
for oldname in de_oldname_with_wrongid:
    

In [36]:
adata_de.obs.target_contrast_gene_name

ENSG00000012963_Stim8hr      UBR7
ENSG00000017260_Stim8hr    ATP2C1
ENSG00000067606_Stim8hr     PRKCZ
ENSG00000092929_Stim8hr    UNC13D
ENSG00000100504_Stim8hr      PYGL
                            ...  
ENSG00000198342_Rest       ZNF442
ENSG00000205572_Rest       SERF1B
ENSG00000221890_Rest        NPTXR
ENSG00000236320_Rest       SLFN14
ENSG00000246705_Rest        H2AFJ
Name: target_contrast_gene_name, Length: 33986, dtype: category
Categories (11527, object): ['A1BG', 'A2M', 'AAAS', 'AACS', ..., 'ZXDC', 'ZYG11B', 'ZYX', 'ZZEF1']

In [35]:
de_oldname_to_newgeneid

,gene_id
target_gene_name_from_sgRNA,
C22orf46,ENSG00000293428
U2AF1L5,ENSG00000275895
C19orf48,ENSG00000293495
MTRNR2L8,MTRNR2L8
MTRNR2L1,MTRNR2L1
CCL3L1,ENSG00000276085
ZNF286B,ENSG00000249459
SEPT6,ENSG00000125354
SGK494,ENSG00000167524


In [27]:
name_df = name_df.set_index('corrected_target_gene_name', drop=False)

In [28]:
name_df

,name,sgRNA,target_gene_name_from_sgRNA,gene_id,corrected_target_gene_name
corrected_target_gene_name,,,,,
ARMC5,ARMC5-1,CTGCCTCGCGCAGCTCGCGG,ARMC5,ENSG00000140691,ARMC5
DDB1,DDB1-1,GGAGTTCGCTGCGCGCTGTT,DDB1,ENSG00000167986,DDB1
FNDC10,FNDC10-1,TGCCCGCTCCCCGCGATCCC,FNDC10,ENSG00000228594,FNDC10
RORC,RORC-1,TCTGTGGGGCCCTGTCCATG,RORC,ENSG00000143365,RORC
SLC35F6,SLC35F6-1,GAACAGCTGGTACTTGGTCC,SLC35F6,ENSG00000213699,SLC35F6
...,...,...,...,...,...
NSD1,NSD1-3,GCAGCCTGCTACCTGAGGGG,NSD1,ENSG00000165671,NSD1
STOML2,STOML2-3,GGATTCTCCGAGGATCCTGA,STOML2,ENSG00000165283,STOML2
ERBIN,ERBIN-3,GATCCAAAGTGACTGATGAA,ERBIN,ENSG00000112851,ERBIN


In [25]:
adata_de[adata_de.obs.target_contrast_gene_name.isin(de_oldname_list)].obs

,target_contrast_gene_name,culture_condition,target_contrast,chunk,n_cells_target,target_contrast_gene_name_corrected
ENSG00000277796_Rest,CCL3L1,Rest,ENSG00000277796,6,1369.0,CCL3L3
ENSG00000182230_Stim48hr,FAM153B,Stim48hr,ENSG00000182230,16,608.0,ENSG00000289731
LUZP6_Stim48hr,LUZP6,Stim48hr,LUZP6,20,183.0,MTPN
SGK494_Stim48hr,SGK494,Stim48hr,SGK494,36,769.0,RSKR
ENSG00000256618_Rest,MTRNR2L1,Rest,ENSG00000256618,38,781.0,MTRNR2L1
ENSG00000167747_Rest,C19orf48,Rest,ENSG00000167747,127,618.0,C19orf48P
ENSG00000184208_Rest,C22orf46,Rest,ENSG00000184208,140,1962.0,C22orf46P
ENSG00000255823_Rest,MTRNR2L8,Rest,ENSG00000255823,140,270.0,MTRNR2L8
SEPT6_Stim8hr,SEPT6,Stim8hr,SEPT6,144,885.0,SEPTIN6
ENSG00000124333_Rest,VAMP7,Rest,ENSG00000124333,154,1117.0,VAMP7


In [20]:
adata_de.obs.loc[adata_de.obs.target_contrast_gene_name_corrected==oldname, 'target_contrast_gene_name_corrected']

ENSG00000184208_Rest        C22orf46
ENSG00000184208_Stim48hr    C22orf46
ENSG00000184208_Stim8hr     C22orf46
Name: target_contrast_gene_name_corrected, dtype: category
Categories (11527, object): ['A1BG', 'A2M', 'AAAS', 'AACS', ..., 'ZXDC', 'ZYG11B', 'ZYX', 'ZZEF1']

In [13]:
adata_de.obs

,target_contrast_gene_name,culture_condition,target_contrast,chunk,n_cells_target,target_contrast_gene_name_corrected
ENSG00000012963_Stim8hr,UBR7,Stim8hr,ENSG00000012963,0,491.0,UBR7
ENSG00000017260_Stim8hr,ATP2C1,Stim8hr,ENSG00000017260,0,469.0,ATP2C1
ENSG00000067606_Stim8hr,PRKCZ,Stim8hr,ENSG00000067606,0,427.0,PRKCZ
ENSG00000092929_Stim8hr,UNC13D,Stim8hr,ENSG00000092929,0,830.0,UNC13D
ENSG00000100504_Stim8hr,PYGL,Stim8hr,ENSG00000100504,0,414.0,PYGL
...,...,...,...,...,...,...
ENSG00000198342_Rest,ZNF442,Rest,ENSG00000198342,680,324.0,ZNF442
ENSG00000205572_Rest,SERF1B,Rest,ENSG00000205572,680,44.0,SERF1B
ENSG00000221890_Rest,NPTXR,Rest,ENSG00000221890,680,293.0,NPTXR
ENSG00000236320_Rest,SLFN14,Rest,ENSG00000236320,680,848.0,SLFN14


In [14]:
de_oldname_to_newname

,target_gene_name_from_sgRNA,corrected_target_gene_name
5,C22orf46,C22orf46P
937,U2AF1L5,ENSG00000275895
1587,C19orf48,C19orf48P
1820,MTRNR2L8,MTRNR2L8
2719,MTRNR2L1,MTRNR2L1
3844,CCL3L1,CCL3L3
5448,ZNF286B,ZNF286B
5464,SEPT6,SEPTIN6
6131,SGK494,RSKR
7323,SMIM11B,SMIM11


In [22]:
name_df

,name,sgRNA,target_gene_name_from_sgRNA,gene_id,corrected_target_gene_name
0,ARMC5-1,CTGCCTCGCGCAGCTCGCGG,ARMC5,ENSG00000140691,ARMC5
1,DDB1-1,GGAGTTCGCTGCGCGCTGTT,DDB1,ENSG00000167986,DDB1
2,FNDC10-1,TGCCCGCTCCCCGCGATCCC,FNDC10,ENSG00000228594,FNDC10
3,RORC-1,TCTGTGGGGCCCTGTCCATG,RORC,ENSG00000143365,RORC
4,SLC35F6-1,GAACAGCTGGTACTTGGTCC,SLC35F6,ENSG00000213699,SLC35F6
...,...,...,...,...,...
25507,NSD1-3,GCAGCCTGCTACCTGAGGGG,NSD1,ENSG00000165671,NSD1
25508,STOML2-3,GGATTCTCCGAGGATCCTGA,STOML2,ENSG00000165283,STOML2
25509,ERBIN-3,GATCCAAAGTGACTGATGAA,ERBIN,ENSG00000112851,ERBIN
25510,ZNF563-3,GGCGGGTCCCACTGTGACAG,ZNF563,ENSG00000188868,ZNF563


In [18]:
set(adata_de.obs.target_contrast) - set(name_df.gene_id)

{'AKAP2',
 'ENSG00000124333',
 'ENSG00000124334',
 'ENSG00000148362',
 'ENSG00000167747',
 'ENSG00000182230',
 'ENSG00000184208',
 'ENSG00000203812',
 'ENSG00000232196',
 'ENSG00000255823',
 'ENSG00000256618',
 'ENSG00000277796',
 'ENSG00000290687',
 'LUZP6',
 'SEPT6',
 'SGK494',
 'SMIM11B',
 'U2AF1L5'}

In [20]:
temp = adata_de[adata_de.obs.target_contrast.isin(list(set(adata_de.obs.target_contrast) - set(name_df.gene_id)))].obs.target_contrast_gene_name.unique()
temp

['CCL3L1', 'FAM153B', 'LUZP6', 'SGK494', 'MTRNR2L1', ..., 'HIST2H2AA3', 'IL9R', 'ZNF286B', 'AKAP2', 'U2AF1L5']
Length: 18
Categories (18, object): ['AKAP2', 'C19orf48', 'C22orf46', 'CCL3L1', ..., 'SMIM11B', 'U2AF1L5', 'VAMP7', 'ZNF286B']

In [25]:
name_df[name_df.target_gene_name_from_sgRNA.isin(temp)]

,name,sgRNA,target_gene_name_from_sgRNA,gene_id,corrected_target_gene_name
5,C22orf46-1,GTCGAAGAAGGAACGTACTT,C22orf46,ENSG00000293428,C22orf46P
937,U2AF1L5-1,CACCGAGAAAGACAAGTGAG,U2AF1L5,ENSG00000275895,ENSG00000275895
1587,C19orf48-1,CTTCGGAGAGAGAAATGCTG,C19orf48,ENSG00000293495,C19orf48P
1820,MTRNR2L8-1,AAAGCTGTTCGGTAGTAAGG,MTRNR2L8,MTRNR2L8,MTRNR2L8
2719,MTRNR2L1-1,GGCTTCTTCTAGGCCAACTT,MTRNR2L1,MTRNR2L1,MTRNR2L1
3844,CCL3L1-1,TGGGAGCCGCCACCAGCAGG,CCL3L1,ENSG00000276085,CCL3L3
5448,ZNF286B-1,TGTGGTGATGTCTCTCGTCT,ZNF286B,ENSG00000249459,ZNF286B
5464,SEPT6-1,GTGGCAGCCCTGGCGGACGC,SEPT6,ENSG00000125354,SEPTIN6
6131,SGK494-1,GAGCAGTAAGCTGTCGGCAG,SGK494,ENSG00000167524,RSKR
7323,SMIM11B-1,CGTCACACACTTACTGCTCC,SMIM11B,ENSG00000205670,SMIM11
